In [ ]:
# !pip3 install snscrape
import pandas as pd
from tqdm.notebook import tqdm
# import snscrape.modules.twitter as sntwitter

In [ ]:
#cleaning the data/preprocess

In [ ]:
import pandas as pd
import numpy as np

import nltk
import re

from wordcloud import WordCloud

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Downloading all the requirements
# !pip install scikit-multilearn
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

# !pip install contractions
import contractions

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, make_union

from tqdm import tqdm

# !pip install -U imbalanced-learn

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

# Setting global parameters for the Plots
rcParams['figure.figsize'] = 18, 9 
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['lines.linewidth'] = 2.5
rcParams['xtick.labelsize'] = 'smaller'
rcParams['ytick.labelsize'] = 'smaller'
rcParams['axes.labelpad'] = 15

# !pip install langdetect

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
class preprocess:

    def __init__(self):
        pass

    def clean(self, df, column):
      
        # applying all the methods in sequence
        df[column] = df[column].apply(self.rm_hash)
        df[column] = df[column].str.lower()
        df[column] = self.fix_contractions(df, column)

        # df[column] = df[column].apply(self.rm_decontracted)
        df[column] = df[column].apply(self.rm_urls)
        df[column] = df[column].apply(self.rm_html_tags)
        df[column] = df[column].apply(self.rm_non_alpha)
        # df[column] = self.rm_stopwords(df, column)

        # df[column] = df[column].apply(self.rm_patterns_removal) #ZZZZZZ REMOVER
        df[column] = df[column].apply(self.rm_repeated_text) ### ZZZZ TO Z
        df[column] = df[column].apply(self.rm_numword_removal)
        df[column] = self.rm_whitespace(df, column)

        # lowercasing the text


        df = self.lemmatization(df, column)
        # df[column] = self.rm_stopwords(df, column)
        # df = self.remove_less_2(df, column)
    
        return df


    def rm_hash(self,text):
        url_pattern = re.compile(r'(\s)#\w+')
        return url_pattern.sub(r'', text)

    def rm_urls(self, text):
        return re.sub(r'http\S+', ' ', text)
    
    def rm_html_tags(self, text):
        clean = re.compile('<.*?>')
        return re.sub(clean, ' ', text)

    def rm_non_alpha(self, text):
        return re.sub('[^a-zA-Z]', ' ', text)

    def rm_whitespace(self, df, column):
        return df[column].apply(lambda x : " ".join(x.split()))

    def lemmatization(self, df, column):
        lemma = WordNetLemmatizer()

        # Lemmatizing with respect to the verb form and noun form
        df[column] = df[column].apply(lambda x : " ".join([lemma.lemmatize(word = w, pos='v') for w in x.split(" ")]))
        df[column] = df[column].apply(lambda x : " ".join([lemma.lemmatize(word = w, pos='n') for w in x.split(" ")]))

        # df[column] = df[column].apply(lambda x : " ".join([lemma.lemmatize(word = w, pos='a') for w in x.split(" ")]))	
        # df[column] = df[column].apply(lambda x : " ".join([lemma.lemmatize(word = w, pos='r') for w in x.split(" ")]))	
        # df[column] = df[column].apply(lambda x : " ".join([lemma.lemmatize(word = w, pos='s') for w in x.split(" ")]))		

        return df

    def rm_stopwords(self, df, column):

        # Generating all the stop words of english and removing no,not nor as they are significantly important
        stop = stopwords.words('english')
        not_stop_words = {'no', 'not', 'nor','very','but'}
        modified_stop = [ele for ele in stop if ele not in not_stop_words]

        return df[column].apply(lambda x : ' '.join([word for word in x.split(' ') if word not in modified_stop]))

    # As the words with less than length 2 will not contribute to toxicity in a general manner
    def _clean_small(self, text):
        return re.sub(r'\b\w{1,2}\b', '', text)

    # After every re.sub we get varied amount of whitespaces so it is essential to remove white spaces
    def remove_less_2(self, df, column):
        df[column] = df[column].apply(self._clean_small)
        df[column] = self.rm_whitespace(df, column)
        return df

    # zzzzzzzzzz remover
    def rm_patterns_removal(self,text): 
        return re.sub("\\s*\\b(?=\\w*(\\w)\\1{2,})\\w*\\b",' ',text)

    #function to remove words with numbers
    def rm_numword_removal(self,text):
        return re.sub("\S*\d\S*", " ", text).strip()

    def rm_decontracted(self,text):
      phrase = re.sub(r"\'t", " not", text)
      phrase = re.sub(r"\'re", " are", phrase)
      phrase = re.sub(r"\'s", " is", phrase)
      phrase = re.sub(r"\'d", " would", phrase)
      phrase = re.sub(r"\'ll", " will", phrase)
      phrase = re.sub(r"\'ve", " have", phrase)
      phrase = re.sub(r"\'m", " am", phrase)
      return phrase

    # zzzzzzzzzz updater to z remover
    def rm_repeated_text(self,text):
      return re.sub(r'(.)\1{2,}', r'\1', text) 

    def fix_contractions(self, df, column):
        return df[column].apply(contractions.fix)

In [ ]:
# df_test = preprocess().clean(input, 'content')
# df_test

,content
0,
1,meet a muslim guy from up want to become ia of...
2,ruko zara sabar karo dhakka mukki nahi karne k...
3,break news up stf hmoindia myogiadityanath upp...
4,pat se headshot cm myogioffice once say mitti ...
5,please for the love of god stay onside and lea...
6,le tr s joli geste de leroy san envers sadio m...
7,thomas amp sadio in today s train bild camo og...
8,leroy sane hide lower lip from the camera obsc...
9,good morning navy blue m xxl kindly dm nationw...


In [ ]:
# from langdetect import detect
# lst=""
# file=[]
# j=0
# for i,x in enumerate(df_test['content']):
#   if(len(x)!=0 and detect(x)=='en'):
#       file.append(x)
# file_df=pd.DataFrame(file,columns=["content"])
# file_df

,content
0,meet a muslim guy from up want to become ia of...
1,ruko zara sabar karo dhakka mukki nahi karne k...
2,break news up stf hmoindia myogiadityanath upp...
3,pat se headshot cm myogioffice once say mitti ...
4,please for the love of god stay onside and lea...
5,le tr s joli geste de leroy san envers sadio m...
6,thomas amp sadio in today s train bild camo og...
7,leroy sane hide lower lip from the camera obsc...
8,good morning navy blue m xxl kindly dm nationw...


In [ ]:
nltk.download('punkt') # one time execution

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2023-04-13 11:02:16--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-13 11:02:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-13 11:02:16--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
# nltk.download('punkt') # one time execution
import re

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

In [ ]:
#defining hashtags for getting tweets

In [ ]:
hash_tag = ["Odisha","Truss",]
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
for x in hash_tag:
  print(x)

Odisha
Truss


In [ ]:
#function for getting top tweets and data

In [ ]:
data_tr=[]
k=0
for x in hash_tag:
    print(x)
    scraper=sntwitter.TwitterSearchScraper(x)
    tweets=[]

    for i,tweet in enumerate(scraper.get_items()):
      data=[
          tweet.date,
          tweet.rawContent,
          tweet.user.username,
          tweet.likeCount,
          tweet.retweetCount
      ]
      tweets.append(data)
      if i > 400:
        break

    tweet_df=pd.DataFrame(tweets,columns=["date","content","username","likecount","retweet"])

    input=pd.DataFrame(tweet_df,columns=["content"])

    df_test = preprocess().clean(input, 'content')

    lst=""
    file=[]
    j=0
    for i,x in enumerate(df_test['content']):
      if(len(x)!=0 and detect(x)=='en'):
          file.append(x)
    file_df=pd.DataFrame(file,columns=["content"])
		
    lst=""
    spa=[]
    j=0
    for i,x in enumerate(file_df['content']):
      lst=lst+str(x)+". "
      #print(x)
      if(i%30==0):
        spa.append(lst)
        lst=""
      
      
    file_df=pd.DataFrame(spa,columns=["content"])
    #print(file_df)

    # sentences = []
    for x in range(0,len(file_df)):
      sentences = []
      sentences=sent_tokenize(file_df.content[x])

      if(len(sentences)<15):
        continue

      #print(len(sentences))
      
      sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
      stop_words = stopwords.words('english')
      sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
      
      
      word_embeddings = {}
      f = open('glove.6B.100d.txt', encoding='utf-8')
      for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
      f.close()
      
      
      sentence_vectors = []
      for i in sentences_clean:
        if len(i) != 0:
          v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
          v = np.zeros((100,))
        sentence_vectors.append(v)
        
        
        # similarity matrix
      sim_mat = np.zeros([len(sentences), len(sentences)])
      
      for i in range(len(sentences)):
        for j in range(len(sentences)):
          if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
          
      
      # from scipy.sparse import coo_array
      nx_graph = nx.from_numpy_array(sim_mat)
      scores = nx.pagerank(nx_graph,max_iter=300,tol=1.0e-4)
      
      
      ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
      
      
      # Specify number of sentences to form the summary
      sn = 10
      temp=""
      #print(len(ranked_sentences))
      # Generate summary
      for i in range(sn):
        # print(ranked_sentences[i][1])
        temp+=ranked_sentences[i][1]
        # print("----------------------------------------------------------------------")
      data_tr.append(temp)


tr=pd.DataFrame(data_tr,columns=["content"])

Odisha
Truss


In [ ]:
tr=pd.DataFrame(data_tr,columns=["content"])

In [ ]:
tr

,content
0,naveen odisha please have a look at pitiful st...
1,chad infi very easy to create agitation there ...
2,odisha congress twist for kannadigas what be y...
3,same modus operandi every where be it bengal o...
4,why appropriate action be not take against the...
5,due to low national medium coverage odisha do ...
6,visakhapatnamjn vskpism chikudasji waltair sha...
7,bbcpm if you have ask just yr ago who run the ...
8,nilegardiner it be not true but even if it be ...
9,gbnews martindaubney and you do not nation tor...


In [ ]:
tr_new=tr

In [ ]:
# frames=[data_app,tr]
# result = pd.concat(frames)
# display(result)

In [ ]:
# # result.to_csv("out.csv")
# tweet_df = pd.read_csv('out.csv')
# tweet_df

,Unnamed: 0,content
0,0,atiq ahmed s son asad ahmed kill in up police ...
1,1,big break news mohammad asad son of gangster p...
2,2,big break news mohammad asad son of gangster p...
3,3,yadavakhilesh mitti me milana shuru asad son o...
4,4,big break news mohammad asad son of gangster p...
...,...,...
73,73,flight callsign have land at london heathrow a...
74,74,now that the british government have apologise...
75,75,flight callsign have land at london heathrow a...
76,76,flight number from london heathrow to miami in...


In [ ]:
# using openAI api

In [ ]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
df_tr = pd.read_csv('test_fin.csv')
df_tr=df_tr.drop(['Unnamed: 0'], axis=1)

In [ ]:
df_tr['content'][1]

'chad infi very easy to create agitation there after all they be follow the footstep of west bengal te orists and naveen patnaik will continue to be lethargic i do not believe the hindu of odisha be even a agile or inform a the hindu of gujarat or up or elsewhere.sasmitpatra dpradhanbjp the people of kotia and koraput deserve a resolution to the longstanding issue not just lip service from dpradhanbjp it be time for him to prove his commitment to odisha by approach the central government and inter state council for a solution.sasmitpatra dpradhanbjp the kotia issue be a matter of concern for all of odisha not just a few politician dpradhanbjp need to step up and take the lead in find a solution through the central government and inter state council.sasmitpatra dpradhanbjp the kotia issue be not just about medium coverage or political drama it be about the welfare of the people of odisha dpradhanbjp need to step up and take concrete step to resolve this issue through the central governm

In [ ]:
#please use you key 

In [ ]:
import openai
openai.api_key='sk-*************************************************'

In [ ]:
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
r = []
for ind in tqdm(range(len(df_tr)),desc="Doin' my work"):
    prompt = "Summarize this in 50 words:"+df_tr['content'][ind]
    completion = openai.Completion.create(
    engine = "text-davinci-003",
    prompt=prompt,
    max_tokens=64,
    n=1,
    stop=None,
    temperature=0.5)
    r.append(completion.choices[0].text)

Doin' my work: 100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


In [ ]:
tweet_df_out = pd.DataFrame(r,columns=["summary"])

In [ ]:
tweet_df_out['summary'][0]

'\n\nLiz Truss has been heavily criticized for her handling of the UK economy, with many claiming she has destroyed it. Her mini budget wiped billions off the economy and hiked mortgage payments for people across the country, yet she still tried to blame others for her downfall. Angela Rayner has pointed out that the'

In [ ]:
result = pd.concat([df_tr, tweet_df_out], axis=1, join='inner')
result.to_csv("sum.csv")

In [ ]:
# result